In [75]:
#Importing libraries
from tweepy import OAuthHandler
from tweepy import Cursor
from tweepy import API
import pandas as pd
import re
from textblob import TextBlob
import plotly.graph_objects as go
from tabulate import tabulate

In [76]:
# Getting twitter Authentication credentials
CONSUMER_KEY="jJyU52WnIin7QcfcwBtBBcQ2m"
CONSUMER_SECRET="z0Bv43nI1LYef4fvmL8hSO0ywYxryMuF5fnYZGvK2KCpDOJXx4"
ACCESS_TOKEN="1382621002832572419-ETl0oJcJxJp3bsDSNwllnfIFjbqk7o"
ACCESS_TOKEN_SECRET="9EaW2JHW1JKEz1e4ig4lblxVXeZ6oXVnetJYNMg0GqT8a"
auth=OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN,ACCESS_TOKEN_SECRET)

# Create API object
api=API(auth,wait_on_rate_limit=True)

# Extracting tweets from twitter

no_tweets=100

bjp_posts1=Cursor(api.search, q='#BJP', lang="en", since="2020-01-01").items(no_tweets)
bjp_posts2=Cursor(api.search, q='#NDA', lang="en", since="2020-01-01").items(no_tweets)
bjp_posts3=Cursor(api.search, q='#Narendra Modi', lang="en", since="2020-01-01").items(no_tweets)


inc_posts1=Cursor(api.search, q='#INC', lang="en", since="2020-01-01").items(no_tweets)
inc_posts2=Cursor(api.search, q='#Congress', lang="en", since="2020-01-01").items(no_tweets)
inc_posts3=Cursor(api.search, q='#Rahul Gandhi', lang="en", since="2020-01-01").items(no_tweets)

In [77]:
#Creating a dataframe with a column called Tweets

bjp_df=pd.DataFrame([tweet.text for tweet in bjp_posts1],columns=['Tweets'])
bjp_df1=pd.DataFrame([tweet.text for tweet in bjp_posts2],columns=['Tweets'])
bjp_df2=pd.DataFrame([tweet.text for tweet in bjp_posts3],columns=['Tweets'])
bjp_df = pd.concat([bjp_df,bjp_df1,bjp_df2], ignore_index=True)

inc_df=pd.DataFrame([tweet.text for tweet in inc_posts1],columns=['Tweets'])
inc_df1=pd.DataFrame([tweet.text for tweet in inc_posts2],columns=['Tweets'])
inc_df2=pd.DataFrame([tweet.text for tweet in inc_posts3],columns=['Tweets'])
inc_df = pd.concat([inc_df,inc_df1,inc_df2], ignore_index=True)

#For Demonstration
bjp_df

,Tweets
0,@PMOIndia @rmulko @JoeBiden @KamalaHarris @PMO...
1,@rmulko @JoeBiden @KamalaHarris @PMOIndia @myo...
2,@ANINewsUP @ANI 😂😂😂😂 @IncomeTaxIndia is the Do...
3,@Ankursingh116 @Aktuchhatrsangh #aktuonlineExa...
4,@htTweets AKTU ONLINE EXAM \nIF OFFLINE EXAMS ...
...,...
247,#98th Annual Function #Vihangam #Yoga is going...
248,Unity needed to set crypto rules - PM Narendra...
249,"""Country's research was misused"" PM #Narendra ..."
250,Narendra Modi to take final decision on regula...


In [78]:
#Removing unusual words and symbols from the Tweets
def cleanTxt(text):
    text=re.sub(r'@[A-Za-z0-9_s:s]+','',text)     # Removed @...
    text=re.sub(r'#[A-Za-z0-9_s:s]+','',text)     # Removed #...
    text=re.sub(r'hmm ','',text)                  # Removed huu
    text=re.sub(r'[Hh]i ','',text)                # Removed Hi
    text=re.sub(r'and ','',text)                  # Removed and
    text=re.sub(r'oh ','',text)                   # Removed oh
    text=re.sub(r'via ','',text)                  # Removed via
    text=re.sub(r'RT[\s]+','',text)               # Removed RT
    text=re.sub(r'https[:s]\/\/\S+','',text)      # Removed https link
    return text

bjp_df['Tweets']=bjp_df['Tweets'].apply(cleanTxt)

inc_df['Tweets']=inc_df['Tweets'].apply(cleanTxt)

#For Demonstration
bjp_df

,Tweets
0,ONLINE EXAM \nIF OFFLINE EXAMS THEN THIS...
1,ONLINE EXAM \nIF OFFLINE EXAMS THEN THIS ...
2,😂😂😂😂 is the Doberman unit of have got t...
3,\n \nStill no one is understanding how…
4,AKTU ONLINE EXAM \nIF OFFLINE EXAMS THEN THIS...
...,...
247,Annual Function is going to be celebrated i...
248,Unity needed to set crypto rules - PM Narendra...
249,"""Country's research was misused"" PM Modi"
250,Narendra Modi to take final decision on regula...


In [79]:
# Creat a fun. to get polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

# Creat a new columns

bjp_df['Polarity']=bjp_df['Tweets'].apply(getPolarity)

inc_df['Polarity']=inc_df['Tweets'].apply(getPolarity)

#For Demonstration
bjp_df

,Tweets,Polarity
0,ONLINE EXAM \nIF OFFLINE EXAMS THEN THIS...,0.00
1,ONLINE EXAM \nIF OFFLINE EXAMS THEN THIS ...,0.00
2,😂😂😂😂 is the Doberman unit of have got t...,0.30
3,\n \nStill no one is understanding how…,0.00
4,AKTU ONLINE EXAM \nIF OFFLINE EXAMS THEN THIS...,0.00
...,...,...
247,Annual Function is going to be celebrated i...,0.35
248,Unity needed to set crypto rules - PM Narendra...,0.00
249,"""Country's research was misused"" PM Modi",0.00
250,Narendra Modi to take final decision on regula...,0.00


In [80]:
# Creat a function to compute the negative, neutral and positive tweets
def getAnalysis(score):
    if score < 0:
        return "Negative"
    elif score == 0:
        return "Neutral"
    else:
        return "Positive"

# Creat a new columns
    
bjp_df['Analysis']=bjp_df['Polarity'].apply(getAnalysis)

inc_df['Analysis']=inc_df['Polarity'].apply(getAnalysis)

# Show the dataframe
bjp_df

,Tweets,Polarity,Analysis
0,ONLINE EXAM \nIF OFFLINE EXAMS THEN THIS...,0.00,Neutral
1,ONLINE EXAM \nIF OFFLINE EXAMS THEN THIS ...,0.00,Neutral
2,😂😂😂😂 is the Doberman unit of have got t...,0.30,Positive
3,\n \nStill no one is understanding how…,0.00,Neutral
4,AKTU ONLINE EXAM \nIF OFFLINE EXAMS THEN THIS...,0.00,Neutral
...,...,...,...
247,Annual Function is going to be celebrated i...,0.35,Positive
248,Unity needed to set crypto rules - PM Narendra...,0.00,Neutral
249,"""Country's research was misused"" PM Modi",0.00,Neutral
250,Narendra Modi to take final decision on regula...,0.00,Neutral


In [81]:
# Show the dataframe
inc_df

,Tweets,Polarity,Analysis
0,"Learn to think long term, you will be forever ...",0.4625,Positive
1,Self Goal - DelConfidential - Indian Express ...,0.0000,Neutral
2,👉 $119.95 👈\nBlakeway Worldwide Panoramas Inc ...,0.0000,Neutral
3,Listen to the stupendous Congress spokesperson...,0.0000,Neutral
4,Would u speak on the crowds for also.. lol ...,0.8000,Positive
...,...,...,...
295,I ask loyalists - who benefited most from Gan...,0.5000,Positive
296,I ask loyalists - who benefited most from Gan...,0.5000,Positive
297,I ask loyalists - who benefited most from Gan...,0.5000,Positive
298,Gandsaid Namo Ji destroyed India with his 20 ...,-0.4000,Negative


In [82]:
# Removing all nutral tweets

bjp_df.drop(bjp_df[bjp_df['Analysis']=='Neutral'].index, inplace=True)

inc_df.drop(inc_df[inc_df['Analysis']=='Neutral'].index, inplace=True)

# Calculating Percentage of positive tweets

a=bjp_df.groupby('Analysis').count();
b=inc_df.groupby('Analysis').count();
print(a)
print(b)

bjp_pos=round(a['Tweets'][1]*100/bjp_df.shape[0])
inc_pos=round(b['Tweets'][1]*100/inc_df.shape[0])

          Tweets  Polarity
Analysis                  
Negative      19        19
Positive     104       104
          Tweets  Polarity
Analysis                  
Negative      20        20
Positive     114       114


In [83]:
# Plotting graph

Party=['B J P','I N C']
pos=[ bjp_pos, inc_pos ]
neg=[ 100-bjp_pos, 100-inc_pos ]

fig=go.Figure(data=[
    go.Bar(name='+ve',x=Party,y=pos),
    go.Bar(name='-ve',x=Party,y=neg)
])

fig.update_layout(title_text="Analysis of Tweets For Both Party",
                  title_font_size=20,
                  barmode='group')
fig.show()

In [84]:
#Calculating no of seats
tot_bjp_accurate=round(bjp_pos*543/(bjp_pos+inc_pos))

#Creating Exit Poll Table
data=[['Party', 'Seats/543'],
      ['BJP', tot_bjp_accurate],
      ['INC', 543-tot_bjp_accurate]]
print('\tEXIT POLL')
print(tabulate(data, headers='firstrow', tablefmt='fancy_grid'))

	EXIT POLL
╒═════════╤═════════════╕
│ Party   │   Seats/543 │
╞═════════╪═════════════╡
│ BJP     │         272 │
├─────────┼─────────────┤
│ INC     │         271 │
╘═════════╧═════════════╛
